In [9]:
import pickle
import numpy as np
import pandas as pd
import itertools
import matplotlib.pyplot as plt
from matplotlib import cm
from datetime import datetime
import os
import geopandas as gpd

def read_pickle(fn):
    data_test = []
    with open(fn, 'rb') as handle:
        try:
            while True:
                data_test.append(pickle.load(handle))
        except EOFError:
            pass
    return data_test

pickle_fmt = '/storage/coda1/p-rbras6/0/njadidoleslam3/projects/stochsm/stage4_analysis/monthly/{month}_new.pickle'
in_pick_fmt = '/storage/coda1/p-rbras6/0/njadidoleslam3/projects/stochsm/stage4_analysis/events/{year}_new.pickle'


In [10]:
# fn_st_grid_masked = '/storage/coda1/p-rbras6/0/njadidoleslam3/projects/stochsm/data/gis_files/st4/stage4_grid_new.shp'
# st4_grid_masked = gpd.read_file(fn_st_grid_masked)



for month in [7]:# range(1,13):
    # month = 5
    in_pickle = pickle_fmt.format(month = month)
    data = read_pickle(in_pickle)
    data_all = []
    for d in data:
        data_all.extend(d)


    events_all = pd.DataFrame(data_all, columns=['grid_xy','tr', 'i', 'tb'])
    events_all['h'] = events_all['i'] * events_all['tr']

    summary = events_all.groupby('grid_xy').mean().reset_index()
    counts = events_all.groupby('grid_xy').count().reset_index()
    var_h = events_all.groupby('grid_xy').var().reset_index() # standard deviation of storm depths
    
    
    summary['lambda'] = summary['h']/var_h['h']
    summary['kappa'] = summary['h']* summary['lambda']
    summary['count'] = counts['tr']/21
    # summary['count'] = summary['count'].astype(np.int32)

    summary_path = '/storage/coda1/p-rbras6/0/njadidoleslam3/projects/stochsm/stage4_analysis/summary'
    if not os.path.exists(summary_path):
        os.makedirs(summary_path)

    fn_summary = os.path.join(summary_path, '{month}_new.pickle'.format(month = month))
    with open(fn_summary, 'wb') as handle:
        pickle.dump(summary, handle, protocol= pickle.HIGHEST_PROTOCOL)

The above code took 1 hour to write summary data into pickles

In [16]:
summary

,grid_xy,tr,i,tb,h,lambda,kappa,count
0,100150506,40.100000,0.115162,688.700000,3.841000,0.168074,0.645571,0.476190
1,100150507,40.100000,0.115162,688.700000,3.841000,0.168074,0.645571,0.476190
2,100160467,43.636364,0.028523,552.000000,1.418182,0.335938,0.476421,0.523810
3,100160468,43.636364,0.028523,552.000000,1.418182,0.335938,0.476421,0.523810
4,100160469,43.636364,0.032614,641.454545,1.549091,0.325306,0.503929,0.523810
...,...,...,...,...,...,...,...,...
464490,110620166,59.236486,1.113697,48.878378,20.255811,0.023946,0.485047,7.047619
464491,110620167,65.567376,0.923923,50.127660,21.977730,0.025965,0.570645,6.714286
464492,110620168,56.572368,1.004530,46.539474,17.731053,0.027902,0.494738,7.238095
464493,110630164,63.180645,1.023788,43.451613,21.892581,0.011059,0.242103,7.380952


In [17]:
var_list = {
    'tr':       {'max':48, 'min':0, 'label': 'Mean Event Duration [$hr$]', 'bw':6,'nbins':8, 'cmap':'Blues' },
    'i':        {'max':5, 'min':0, 'label': 'Mean Intensity [$mm/hr$]', 'bw':1,'nbins':10,'cmap':'Blues'},
    'tb':       {'max':360, 'min':0, 'label': 'Mean Interarrival Time [$hr$]', 'bw':12, 'nbins':15, 'cmap':'Reds'},
    'h':        {'max':20, 'min':0, 'label': 'Mean Storm Depth [$mm$]', 'bw':4,'nbins':20, 'cmap':'Blues'},
    'count':    {'max':10, 'min':0, 'label': 'Mean Number of Events', 'bw':1,'nbins':10, 'cmap':'Blues'}}
# '''
# 'lambda':   {'max':0.26, 'min':0, 'label': 'RMSE [$cm^3/cm^3$]',     'bw':0.01, 'nbins':13, 'cmap':'YlOrRd' },
# 'kappa':    {'max':0.25, 'min':0,  'label': 'MAE [$cm^3/cm^3$]',     'bw':0.01,'nbins':13,'cmap':'Greens'},
# 'cv_tb':    {'max':1.2, 'min':0.8, 'label': '$CV_{tb}$', 'bw':.04,'nbins':10, 'cmap':'RdBu' }}'''

In [12]:
summary_path = '/storage/coda1/p-rbras6/0/njadidoleslam3/projects/stochsm/stage4_analysis/summary'
fn_st_grid_masked = '/storage/coda1/p-rbras6/0/njadidoleslam3/projects/stochsm/data/gis_files/st4/stage4_grid_new.shp'
st4_grid_masked = gpd.read_file(fn_st_grid_masked)

In [13]:
fn_us_states = '/storage/coda1/p-rbras6/0/njadidoleslam3/projects/stochsm/data/gis_files/conus/conus_states.geojson'
us_states = gpd.read_file(fn_us_states)
us_states = us_states.to_crs(4326)

In [18]:
# variable = 'tb'
from matplotlib import ticker

xlim = ([us_states.total_bounds[0],  us_states.total_bounds[2]])
ylim = ([us_states.total_bounds[1],  us_states.total_bounds[3]])

pickle_fmt = '/storage/coda1/p-rbras6/0/njadidoleslam3/projects/stochsm/stage4_analysis/summary/{month}_new.pickle'

for month in [7]:#range(1,13):
    in_pickle = pickle_fmt.format(month = month)
    summary = read_pickle(in_pickle)
    summary = summary[0]
    summary['grid_xy'] = summary['grid_xy'].astype(np.int64)
    data = st4_grid_masked.merge(summary)
    for variable in ['tr']:#list(var_list.keys()):
            


        out_pth = '/storage/coda1/p-rbras6/0/njadidoleslam3/projects/stochsm/figures/st4/summary/{variable}/'.format(variable = variable)
        out_fn = '{variable}_{month}_new1.jpg'.format(variable = variable, month=month)

        if not os.path.exists(out_pth):
            os.makedirs(out_pth)

        fn_temp = '{variable}.png'
        cm1 = plt.cm.get_cmap(var_list[variable]['cmap'],var_list[variable]['nbins']-0)
        fig, ax = plt.subplots(figsize=(20, 16))

        a = data.plot(ax=ax, column=variable, edgecolor='none', cmap=cm1, vmin = var_list[variable]['min'], vmax=var_list[variable]['max'], zorder = 0)
        us_states.plot(ax=ax, facecolor="none", edgecolor="black", zorder = 1, alpha=0.2)
        
        ax.set_axis_off()
        dt_str = '2020-{month}-1'.format(month = month)
        mnth_name = pd.to_datetime(dt_str).month_name()
        ax.set_xlim(xlim)
        ax.set_ylim(ylim)

        ax.text(0.42,-0.03, mnth_name,transform=ax.transAxes, fontsize=30)
        cax = fig.add_axes([0.1, 0.15, 0.8, 0.04])
        fig.colorbar( a.collections[0], cax=cax, orientation='horizontal')
        cax.set_xlabel(var_list[variable]['label'], fontsize=20)

        cax.tick_params(labelsize=18)
        
        x_d = np.linspace(var_list[variable]['min'], var_list[variable]['max'], var_list[variable]['nbins']+1)
        
        labels = []
        for i,x in enumerate(x_d):
            if i%2==0:
                if variable in ['tb', 'tr', 'count']:
                    labels.append(str(int(x)))
                else:
                    labels.append(str(x))
            else:
                labels.append('')


        cax.xaxis.set_major_locator(ticker.FixedLocator(x_d))
        cax.xaxis.set_major_formatter(ticker.FixedFormatter(labels))
        # cax.set_xticks(x_d)
        # cax.set_xticklabels([str(x) for x in x_d])
        fn_out = os.path.join(out_pth,out_fn)
        fig.savefig(fn_out, dpi=300, bbox_inches='tight')
        plt.close(fig)

In [6]:
from matplotlib import ticker

xlim = ([us_states.total_bounds[0],  us_states.total_bounds[2]])
ylim = ([us_states.total_bounds[1],  us_states.total_bounds[3]])

pickle_fmt = '/storage/coda1/p-rbras6/0/njadidoleslam3/projects/stochsm/stage4_analysis/summary/{month}.pickle'

for month in [7]:
    in_pickle = pickle_fmt.format(month = month)
    summary = read_pickle(in_pickle)
    summary = summary[0]
    summary['grid_xy'] = summary['grid_xy'].astype(np.int64)
    data = st4_grid_masked.merge(summary)

In [8]:
data.to_file('/storage/coda1/p-rbras6/0/njadidoleslam3/projects/stochsm/trash/july.geojson')